In [254]:
import pandas as pd
import sqlite3
import plotly.graph_objects as go
import numpy as np

## Подключение к базе данных

In [255]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

## Анализировать только пользователей, а не администраторов, учитывать только логи из таблицы проверки, где статус готов

In [256]:
query = '''
SELECT
    timestamp,
    COUNT(*) AS commits,
    uid
FROM
    checker
WHERE
    status = 'ready'
    AND uid LIKE 'user_%'
    AND labname = 'project1'
GROUP BY
    uid, timestamp
'''
df = pd.read_sql(query, conn)

In [257]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = df['timestamp'].dt.date

# Группировка по пользователю и дате
daily_df = df.groupby(['uid', 'date']).agg({'commits': 'sum'}).reset_index()

# Сортировка и накопительное суммирование коммитов по пользователю
daily_df['cumulative_commits'] = daily_df.groupby('uid')['commits'].cumsum()

In [258]:
df

,timestamp,commits,uid,date
0,2020-05-14 20:56:08.898880,1,user_1,2020-05-14
1,2020-05-14 20:58:02.313690,1,user_1,2020-05-14
2,2020-05-14 20:58:46.322457,1,user_1,2020-05-14
3,2020-05-14 21:08:15.434237,1,user_1,2020-05-14
4,2020-05-14 21:10:14.867603,1,user_1,2020-05-14
...,...,...,...,...
946,2020-05-14 16:18:00.528528,1,user_8,2020-05-14
947,2020-05-14 16:41:03.916147,1,user_8,2020-05-14
948,2020-05-14 17:40:47.781946,1,user_8,2020-05-14
949,2020-05-14 17:46:54.469822,1,user_8,2020-05-14


In [259]:
first_commit_points = []

for user in daily_df['uid'].unique():
    first_date = daily_df[daily_df['uid'] == user]['date'].min()
    zero_point_date = pd.to_datetime(first_date) - pd.Timedelta(days=1)

    first_commit_points.append({
        'uid': user,
        'date': zero_point_date.date(),
        'commits': 0,
        'cumulative_commits': 0
    })

# Объединяем и сортируем
daily_df = pd.concat([daily_df, pd.DataFrame(first_commit_points)], ignore_index=True)
daily_df = daily_df.sort_values(by=['uid', 'date'])

In [260]:
users = df['uid'].unique()
timestamps = sorted(daily_df['date'].unique())

In [261]:
frames = []

for d in timestamps:
    frame_data = daily_df[daily_df['date'] <= d]
    traces = []

    for user in users:
        user_data = frame_data[frame_data['uid'] == user]
        trace = go.Scatter(
            x=user_data["date"],
            y=user_data["cumulative_commits"],
            mode="lines+markers",
            name=user
        )
        traces.append(trace)

    frames.append(go.Frame(data=traces, name=str(d)))

In [262]:
initial_traces = []
for user in users:
    user_data = daily_df[(daily_df['uid'] == user) & (daily_df['date'] == daily_df['date'].min())]
    trace = go.Scatter(
        x=user_data['date'],
        y=user_data['cumulative_commits'],
        mode='lines+markers',
        name=user
    )
    initial_traces.append(trace)

In [263]:
fig = go.Figure(
    data=initial_traces,
    layout=go.Layout(
        title = 'Dynamic of commits per user in project1',
        xaxis = dict(title='Timestamp'),
        yaxis = dict(title='Number of Trials'),
        width=1000,      # ширина в пикселях
        height=600,      # высота в пикселях
        updatemenus=[{
            'buttons':[
                {
                    'args': [None, {'frame': {'duration': 60, 'redraw': True},
                                    'fromcurrent': True}],
                    'label': 'Play',
                    'method': 'animate'
                },
                {
                    'args': [[None], {'frame': {'duration': 0, 'redraw': True},
                                      'mode': 'immediate',
                                      'transition': {'duration': 0}}],
                    'label': 'Pause',
                    'method': 'animate'
                }
            ],
            'direction': 'left',
            'pad': {'r': 10, 't': 87},
            'showactive': False,
            'type': 'buttons',
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top'
        }]
    ),
    frames=frames
)

fig.show()

## Закрыть соединение

In [264]:
conn.close()